# スコアリングフェーズにおけるデータ処理（解決編）

前編において、スコアリングデータのone-hotエンコーディング後に、以下３つの問題が生じる得ることを確認しました。<br>
- モデルデータにないカラムが生成される可能性
- モデルデータにあったカラムが消える可能性
- データ型の違いが理由で上記を生じさせてしまう可能性

そこでこの問題が生じないよう、コードを補強していくことにしましょう。前提として、私達はどのデータが連続変数で、どのデータがカテゴリ変数かは事前に把握していることとします。そして今回のカテゴリ変数はDependents, Gender, Married, Education, Self_Employed, Property_Areaの６変数としましょう。まず最初の補強は、フラットファイルを読み込む段階で、カテゴリ変数をobject型として明記して読み込むことです。

In [1]:
# import sample data: Loan screening data for classification 
import pandas as pd

df = pd.read_csv('../data/av_loan_u6lujuX_CVtuZ9i.csv', header=0, \
                 dtype={'Dependents':object,
                        'Gender':object,
                        'Married':object,
                        'Education':object,
                        'Self_Employed':object,
                        'Property_Area':object})
X  = df.iloc[:, :-1]            # 最終列が審査結果のため最終列以前を特徴量Xとして読込
ID = X.iloc[:, [0]]             # 最初列がPK（Loan_ID）なのでID情報としてセット
X  = X.drop('Loan_ID', axis=1)  # 1列目(Loan_ID)は特徴量ベクトルから削除(drop関数におけるaxis=1は列方向)
y  = df.iloc[:, [-1]]           # 最終列を正解データとして読込

# check the shape
print('----------------------------------------------------------------------------------------')
print('Raw shape: (%i,%i)' %df.shape)
print('ID shape: (%i,%i)' %ID.shape)
print('X shape: (%i,%i)' %X.shape)
print('y shape: (%i,%i)' %y.shape)
print('----------------------------------------------------------------------------------------')
print(X.dtypes)
print('----------------------------------------------------------------------------------------')
print('Check the null count of the target variable: %i' % y.isnull().sum())
print('----------------------------------------------------------------------------------------')

# ローン審査でNOとなったサンプルを1（正例）として変換
class_mapping = {'N':1, 'Y':0}
y_new = y.copy()
y_new.loc[:,'Loan_Status'] = y_new['Loan_Status'].map(class_mapping)
print(y_new.groupby(['Loan_Status']).size())

----------------------------------------------------------------------------------------
Raw shape: (614,13)
ID shape: (614,1)
X shape: (614,11)
y shape: (614,1)
----------------------------------------------------------------------------------------
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
dtype: object
----------------------------------------------------------------------------------------
Check the null count of the target variable: 0
----------------------------------------------------------------------------------------
Loan_Status
0    422
1    192
dtype: int64


続けて、モデリング段階のone-hotエンコーディングを行います。ここに変更はありません。

In [2]:
ohe_columns = ['Dependents','Gender','Married','Education','Self_Employed','Property_Area']
X_ohe = pd.get_dummies(X, dummy_na=True, columns=ohe_columns)
print('X_ohe shape:(%i,%i)' % X_ohe.shape)
X_ohe.head()

X_ohe shape:(614,26)


ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0         NaN             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   
3             2583             2358.0       120.0             360.0   
4             6000                0.0       141.0             360.0   

   Credit_History  Dependents_0  Dependents_1  Dependents_2  Dependents_3+  \
0             1.0             1             0             0              0   
1             1.0             0             1             0              0   
2             1.0             1             0             0              0   
3             1.0             1             0             0              0   
4             1.0             1             0             0              0   

   Dependents_nan        ...          Education_Graduate  \
0               0        ...                           1   
1               0        ...                           1   
2               0        ...                           1   
3               0        ...                           0   
4               0        ...                           1   

   Education_Not Graduate  Education_nan  Self_Employed_No  Self_Employed_Yes  \
0                       0              0                 1                  0   
1                       0              0                 1                  0   
2                       0              0                 0                  1   
3                       1              0                 1                  0   
4                       0              0                 1                  0   

   Self_Employed_nan  Property_Area_Rural  Property_Area_Semiurban  \
0                  0                    0                        0   
1                  0                    1                        0   
2                  0                    0                        0   
3                  0                    0                        0   
4                  0                    0                        0   

   Property_Area_Urban  Property_Area_nan  
0                    1                  0  
1                    0                  0  
2                    1                  0  
3                    1                  0  
4                    1                  0  

[5 rows x 26 columns]

続けて、連続変数の欠損を平均値で置き換えます。ここも変更はありません。

In [3]:
from sklearn.preprocessing import Imputer

# 欠損値NaNを平均値(mean)で置換
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(X_ohe)

# 学習済みImputerを適用しX_newの欠損値を置換
X_ohe_columns = X_ohe.columns.values
X_ohe = pd.DataFrame(imp.transform(X_ohe), columns=X_ohe_columns)

# 結果表示
X_ohe.head()

ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0           5849.0                0.0  146.412162             360.0   
1           4583.0             1508.0  128.000000             360.0   
2           3000.0                0.0   66.000000             360.0   
3           2583.0             2358.0  120.000000             360.0   
4           6000.0                0.0  141.000000             360.0   

   Credit_History  Dependents_0  Dependents_1  Dependents_2  Dependents_3+  \
0             1.0           1.0           0.0           0.0            0.0   
1             1.0           0.0           1.0           0.0            0.0   
2             1.0           1.0           0.0           0.0            0.0   
3             1.0           1.0           0.0           0.0            0.0   
4             1.0           1.0           0.0           0.0            0.0   

   Dependents_nan        ...          Education_Graduate  \
0             0.0        ...                         1.0   
1             0.0        ...                         1.0   
2             0.0        ...                         1.0   
3             0.0        ...                         0.0   
4             0.0        ...                         1.0   

   Education_Not Graduate  Education_nan  Self_Employed_No  Self_Employed_Yes  \
0                     0.0            0.0               1.0                0.0   
1                     0.0            0.0               1.0                0.0   
2                     0.0            0.0               0.0                1.0   
3                     1.0            0.0               1.0                0.0   
4                     0.0            0.0               1.0                0.0   

   Self_Employed_nan  Property_Area_Rural  Property_Area_Semiurban  \
0                0.0                  0.0                      0.0   
1                0.0                  1.0                      0.0   
2                0.0                  0.0                      0.0   
3                0.0                  0.0                      0.0   
4                0.0                  0.0                      0.0   

   Property_Area_Urban  Property_Area_nan  
0                  1.0                0.0  
1                  0.0                0.0  
2                  1.0                0.0  
3                  1.0                0.0  
4                  1.0                0.0  

[5 rows x 26 columns]

最後に、RFEによる特徴量選択を実施します。

In [4]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingClassifier

selector = RFE(GradientBoostingClassifier(random_state=1), n_features_to_select=10, step=.05)
selector.fit(X_ohe, y.as_matrix().ravel())

X_fin = X_ohe.loc[:, X_ohe_columns[selector.support_]]
print('X_fin shape:(%i,%i)' % X_fin.shape)
X_fin.head()

X_fin shape:(614,10)


ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0           5849.0                0.0  146.412162             360.0   
1           4583.0             1508.0  128.000000             360.0   
2           3000.0                0.0   66.000000             360.0   
3           2583.0             2358.0  120.000000             360.0   
4           6000.0                0.0  141.000000             360.0   

   Credit_History  Dependents_1  Married_No  Married_nan  Self_Employed_nan  \
0             1.0           0.0         1.0          0.0                0.0   
1             1.0           1.0         0.0          0.0                0.0   
2             1.0           0.0         0.0          0.0                0.0   
3             1.0           0.0         0.0          0.0                0.0   
4             1.0           0.0         1.0          0.0                0.0   

   Property_Area_Semiurban  
0                      0.0  
1                      0.0  
2                      0.0  
3                      0.0  
4                      0.0

さて、以上でモデリング段階の処理が終わり、ここからがテストデータを読み込みとなります。ここでもカテゴリ変数の明示的な指定をします。

In [6]:
import pandas as pd

# import sample data
# Loan screening data for classification 
df_s = pd.read_csv('../data/av_loan_test_Y3wMUE5_7gLdaTN.csv', header=0, \
                   dtype={'Dependents':object,
                        'Gender':object,
                        'Married':object,
                        'Education':object,
                        'Self_Employed':object,
                        'Property_Area':object})
ID_s = df_s.iloc[:, [0]]            # 最初列がPK（Loan_ID）なのでID情報としてセット
X_s  = df_s.drop('Loan_ID', axis=1) # Loan_IDはKey情報なので特徴量ベクトルから削除(drop関数におけるaxis=1は列方向)

# check the shape
print('----------------------------------------------------------------------------------------')
print('Raw shape: (%i,%i)' %df_s.shape)
print('ID shape: (%i,%i)' %ID_s.shape)
print('X shape: (%i,%i)' %X_s.shape)
print(X_s.dtypes)
ID_s.join(X_s).head()

----------------------------------------------------------------------------------------
Raw shape: (333,12)
ID shape: (333,1)
X shape: (333,11)
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome      int64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
dtype: object


Loan_ID Gender Married Dependents     Education Self_Employed  \
0  LP001015   Male     Yes          0      Graduate            No   
1  LP001022   Male     Yes          1      Graduate            No   
2  LP001031   Male     Yes          2      Graduate            No   
3  LP001035   Male     Yes          2      Graduate            No   
4  LP001051   Male      No          0  Not Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5720                  0       110.0             360.0   
1             3076               1500       126.0             360.0   
2             5000               1800       208.0             360.0   
3             2340               2546       100.0             360.0   
4             3276                  0        78.0             360.0   

   Credit_History Property_Area  
0             1.0         Urban  
1             1.0         Urban  
2             1.0         Urban  
3             NaN         Urban  
4             1.0         Urban

カテゴリ変数をきちんとデータ型でも揃えた状態で、one-hotエンコーディングを実施します。Dependentsに見られた不整合が消えています。

In [7]:
ohe_columns = ['Dependents','Gender','Married','Education','Self_Employed','Property_Area']
X_ohe_s = pd.get_dummies(X_s, dummy_na=True, columns=ohe_columns)
print('X_ohe_s shape:(%i,%i)' % X_ohe_s.shape)
X_ohe_s.head()

X_ohe_s shape:(333,26)


ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5720                  0       110.0             360.0   
1             3076               1500       126.0             360.0   
2             5000               1800       208.0             360.0   
3             2340               2546       100.0             360.0   
4             3276                  0        78.0             360.0   

   Credit_History  Dependents_0  Dependents_1  Dependents_2  Dependents_nan  \
0             1.0             1             0             0               0   
1             1.0             0             1             0               0   
2             1.0             0             0             1               0   
3             NaN             0             0             1               0   
4             1.0             1             0             0               0   

   Gender_Female        ...          Education_Graduate  \
0              0        ...                           1   
1              0        ...                           1   
2              0        ...                           1   
3              0        ...                           1   
4              0        ...                           0   

   Education_Not Graduate  Education_nan  Self_Employed_No  Self_Employed_Yes  \
0                       0              0                 1                  0   
1                       0              0                 1                  0   
2                       0              0                 1                  0   
3                       0              0                 1                  0   
4                       1              0                 1                  0   

   Self_Employed_nan  Property_Area_Rural  Property_Area_Semiurban  \
0                  0                    0                        0   
1                  0                    0                        0   
2                  0                    0                        0   
3                  0                    0                        0   
4                  0                    0                        0   

   Property_Area_Urban  Property_Area_nan  
0                    1                  0  
1                    1                  0  
2                    1                  0  
3                    1                  0  
4                    1                  0  

[5 rows x 26 columns]

スコアリングのone-hotエンコーディングを終えたので、この時点の特徴量リストをモデリング時とスコアリング時で再度比較してみます。

In [9]:
cols_model = set(X_ohe.columns.values)
cols_score = set(X_ohe_s.columns.values)

# モデルにはあったがスコアにはないデータ項目
diff1 = cols_model - cols_score
print('モデルのみに存在する項目: %s' % diff1)

# スコアにはあるがモデルになかったデータ項目
diff2 = cols_score - cols_model
print('スコアのみに存在する項目: %s' % diff2)

モデルのみに存在する項目: {'Dependents_3+'}
スコアのみに存在する項目: {'Gender_Unknown'}


データ型の違いによる不一致は消え、以下の違いだけが残りました。
- Dependents_3+はスコアリングデータには存在していない
- Gender_Unknownはスコアリングデータで新たに登場した

後者はモデリング時点にないカラムなので削除する他ありません。前者はモデル時点の再現のためデータ項目として再登場させなくてはいけません。それを実現する一つの方法が、データフレームを縦に結合する処理(concat)です。一旦、サンプルデータからはなれ、concatの基本動作を確認します。

In [10]:
# カラム構成が同じデータフレームの結合
df1 = pd.DataFrame([[1,2,3]], columns=['c1','c2','c3'])
df2 = pd.DataFrame([[3,2,1]], columns=['c1','c2','c3'])
df_all = pd.concat([df1, df2])
df_all

c1  c2  c3
0   1   2   3
0   3   2   1

In [11]:
# カラム構成が異なるデータフレームの結合
df3 = pd.DataFrame([[0,1,2,3]], columns=['c0','c1','c3','c4'])
df_all = pd.concat([df_all, df3])
df_all

c0  c1   c2  c3   c4
0  NaN   1  2.0   3  NaN
0  NaN   3  2.0   1  NaN
0  0.0   1  NaN   2  3.0

上記のc0とc4は、元々あったデータフレーム（df_all）には存在しなかった項目という点で、スコアリングデータに初めて登場した項目と言えます。一方、c2はモデリングデータにはあったがスコアリングデータでは登場しなかったデータ項目となります。よって対応は、①c0とc4はドロップ、②c2はゼロ補完が妥当と結論されます。

それではサンプルデータに戻ります。モデリング時点のone-hotエンコーディング処理後のカラム構成は、X_ohe_columnsでした。

In [12]:
print(X_ohe_columns)

['ApplicantIncome' 'CoapplicantIncome' 'LoanAmount' 'Loan_Amount_Term'
 'Credit_History' 'Dependents_0' 'Dependents_1' 'Dependents_2'
 'Dependents_3+' 'Dependents_nan' 'Gender_Female' 'Gender_Male'
 'Gender_nan' 'Married_No' 'Married_Yes' 'Married_nan' 'Education_Graduate'
 'Education_Not Graduate' 'Education_nan' 'Self_Employed_No'
 'Self_Employed_Yes' 'Self_Employed_nan' 'Property_Area_Rural'
 'Property_Area_Semiurban' 'Property_Area_Urban' 'Property_Area_nan']


このカラム構成だけを持った（データ部分は持たない）データフレームを作ります。

In [13]:
df_cols_m = pd.DataFrame(None, columns=X_ohe_columns, dtype=float)
df_cols_m

Empty DataFrame
Columns: [ApplicantIncome, CoapplicantIncome, LoanAmount, Loan_Amount_Term, Credit_History, Dependents_0, Dependents_1, Dependents_2, Dependents_3+, Dependents_nan, Gender_Female, Gender_Male, Gender_nan, Married_No, Married_Yes, Married_nan, Education_Graduate, Education_Not Graduate, Education_nan, Self_Employed_No, Self_Employed_Yes, Self_Employed_nan, Property_Area_Rural, Property_Area_Semiurban, Property_Area_Urban, Property_Area_nan]
Index: []

[0 rows x 26 columns]

上記データフレームに対して、スコアリング時点のone-hotエンコーディング後のデータを縦に結合します。

In [ ]:
X_ohe_s2 = pd.concat([df_cols_m, X_ohe_s])
X_ohe_s2.head()

次に、スコアリングデータのみに登場するデータ項目を削除しましょう。

In [ ]:
X_ohe_s2 = X_ohe_s2.drop(list(set(X_ohe_s.columns.values)-set(X_ohe.columns.values)), axis=1)
X_ohe_s2.head()

次に、スコアリングでは登場しなかったデータ項目をゼロ埋めします。Depend_3+がNaNから全てゼロとなりました。

In [ ]:
X_ohe_s2.loc[:,list(set(X_ohe.columns.values)-set(X_ohe_s.columns.values))] = \
    X_ohe_s2.loc[:,list(set(X_ohe.columns.values)-set(X_ohe_s.columns.values))].fillna(0, axis=1)
X_ohe_s2.head()

次に、モデリング時点のデータ項目の並び順を明示的に担保します。以下の通り、reindex_axisを使うことで並び順を制御できます。

In [ ]:
test = pd.DataFrame([[1,2,3]], columns=['c1','c2','c3'])
test = test.reindex_axis(['c2','c3','c1'], axis=1)
test

ここでは、モデリング時点のone-hotエンコーディング後の並び順に制御します。

In [ ]:
X_ohe_s2 = X_ohe_s2.reindex_axis(X_ohe.columns.values, axis=1)
X_ohe_s2.head()

以上で、スコアリング時のone-hotエンコーディングをモデリング時の状況と合致させられました。<br>最後は、モデリング時に各変数の平均値を学習させたImputerを適用すれば、連続変数の欠損値も（学習時の）平均値で置き換えられます。<br>ここまで整合させて初めて（学習済みの）Imputerを適用（transform）できる点が理解できていれば大丈夫です。

In [ ]:
X_ohe_s3 = pd.DataFrame(imp.transform(X_ohe_s2), columns=X_ohe_columns)
X_ohe_s3.head()

RFEによって選択された変数の位置はsupport_属性から取得できたので、スコアリングデータの特徴量の最終形は以下のようになります。

In [ ]:
X_fin_s = X_ohe_s3.loc[:, X_ohe_columns[selector.support_]]
print(X_fin_s.shape)
X_fin_s.head()

以上で、スコアリング段階におけるデータの前処理が終了です。このデータを未知のXとして、学習済みモデルに入力すれば予測値を得ることができます。